# Deep Features for Image Retrieval

In [1]:
import graphlab as gl

## Load again the CIFAR-10 dataset

In [5]:
# we only need the train data, not test data, for clustering / retrieval
image_train = gl.SFrame('image_train_data/')

In [6]:
image_train.head()

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242871761322,1.09545373917, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.525087952614, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566015958786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.12979578972, 0.0, 0.0,0.778194487095, 0.0, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.71786928177, 0.0, 0.0,0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.57818555832, 0.0, 0.0,0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.220677852631, 0.0, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.23753464222, 0.0,0.0, 0.0, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.5737862587, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.658935725689, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


### It is going to be similar to the document retrieval in week 3 (Wikipedia VIPs), but here using the deep features


## Train a nearest-neighbors model for retrieving images using deep features

In [8]:
knn_model = gl.nearest_neighbors.create(image_train,
                                       features=['deep_features'],
                                       label='id')

Starting brute force nearest neighbors model training.

## Use image retrieval model with deep features to find similar images

In [13]:
gl.canvas.set_target('browser')
cat = image_train[18:19]
cat['image'].show() # cat

Canvas is updated and available in a tab in the default browser.


#### Now find a pic that is similar to the above pic of the cat

In [14]:
knn_model.query(cat)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 6.779ms      |

| Done         |         | 100         | 104.846ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,384,0.0,1
0,6910,36.9403137951,2
0,39777,38.4634888975,3
0,36870,39.7559623119,4
0,41734,39.7866014148,5


In [17]:
def get_images_ids(query_result):
    """ helpfer function to return the actual img by reference_label/id """
    return image_train.filter_by(query_result['reference_label'], 'id')

In [18]:
cat_neighbors = get_images_ids(knn_model.query(cat))

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 11.635ms     |

| Done         |         | 100         | 109.222ms    |

+--------------+---------+-------------+--------------+

In [20]:
cat_neighbors.show()

Canvas is updated and available in a tab in the default browser.


In [22]:
car = image_train[8:9]
car['image'].show() # red car

Canvas is updated and available in a tab in the default browser.


In [23]:
get_images_ids(knn_model.query(car))['image'].show() # select only the img column of the SFrame

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 9.684ms      |

| Done         |         | 100         | 103.807ms    |

+--------------+---------+-------------+--------------+

Canvas is updated and available in a tab in the default browser.


### Lambda function to find an show nearest neighbor images

In [26]:
show_neighbors = lambda i: get_images_ids(knn_model.query(image_train[i:i+1]))['image'] .show()
# lambda function to show nearest neighbors for img i

In [27]:
show_neighbors(8)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 15.745ms     |

| Done         |         | 100         | 110.916ms    |

+--------------+---------+-------------+--------------+

Canvas is updated and available in a tab in the default browser.


In [28]:
show_neighbors(26)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 8.651ms      |

| Done         |         | 100         | 104.968ms    |

+--------------+---------+-------------+--------------+

Canvas is updated and available in a tab in the default browser.


In [29]:
show_neighbors(112)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 8.563ms      |

| Done         |         | 100         | 109.626ms    |

+--------------+---------+-------------+--------------+

Canvas is updated and available in a tab in the default browser.
